In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
cd = pd.read_csv("/Users/sls/Documents/GitHub/DL_projects/ANNchrun/Churn_Modelling.csv")
cd.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
cd_original = cd.copy()

In [5]:
#Preprocessing
##dropping irrelevant features
cd = cd.drop(["RowNumber","CustomerId", "Surname"], axis =1)
cd.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
##encoding categorical variables
le_gender = LabelEncoder()
cd["Gender"] = le_gender.fit_transform(cd["Gender"])
cd["Gender"]

0       0
1       0
2       0
3       0
4       0
       ..
9995    1
9996    1
9997    0
9998    1
9999    0
Name: Gender, Length: 10000, dtype: int64

In [11]:
cd["Gender"].value_counts()

Gender
1    5457
0    4543
Name: count, dtype: int64

In [45]:
type(cd["Gender"])

pandas.core.series.Series

In [21]:
##Encoding Geography column
from sklearn.preprocessing import OneHotEncoder
ohe_geo = OneHotEncoder()
geo_ohencoder = ohe_geo.fit_transform(cd[["Geography"]])
geo_ohencoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [22]:
type(geo_ohencoder)

scipy.sparse._csr.csr_matrix

In [39]:
geo_ohencoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [33]:
ohe_geo.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [40]:
geo_ohencoded_df = pd.DataFrame(geo_ohencoder.toarray(), columns = ohe_geo.get_feature_names_out(["Geography"]))

In [42]:
type(geo_ohencoded_df)

pandas.core.frame.DataFrame

In [38]:
geo_ohencoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [44]:
cd = pd.concat([cd.drop("Geography", axis=1), geo_ohencoded_df], axis=1)
cd.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [47]:
#Save the encoders and scalers
with open('le_gender.pkl', 'wb') as file:
    pickle.dump(le_gender, file)

with open('ohe_geo.pkl', 'wb') as file:
    pickle.dump(ohe_geo, file)

In [46]:
type(le_gender), type(ohe_geo) 

(sklearn.preprocessing._label.LabelEncoder,
 sklearn.preprocessing._encoders.OneHotEncoder)

In [58]:
#Creating Independent and Dependent features:

##Independent features
X = cd.drop("Exited", axis = 1)
##Dependent feature
y = cd["Exited"]

In [63]:
X.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0.0,0.0,1.0


In [60]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [65]:
#X is a matrix and y is a vestor
type(X), type(y)

(pandas.core.frame.DataFrame, pandas.core.series.Series)

In [66]:
#Splittign into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

#Scaling the feature
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [67]:
type(X_train), type(X_test), type(y_train)

(numpy.ndarray, numpy.ndarray, pandas.core.series.Series)

In [72]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [73]:
cd

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [74]:
cd_original.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


##ANN Implementation

In [76]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [77]:
cd_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [78]:
cd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        10000 non-null  int64  
 1   Gender             10000 non-null  int64  
 2   Age                10000 non-null  int64  
 3   Tenure             10000 non-null  int64  
 4   Balance            10000 non-null  float64
 5   NumOfProducts      10000 non-null  int64  
 6   HasCrCard          10000 non-null  int64  
 7   IsActiveMember     10000 non-null  int64  
 8   EstimatedSalary    10000 non-null  float64
 9   Exited             10000 non-null  int64  
 10  Geography_France   10000 non-null  float64
 11  Geography_Germany  10000 non-null  float64
 12  Geography_Spain    10000 non-null  float64
dtypes: float64(5), int64(8)
memory usage: 1015.8 KB


In [81]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8000, 12), (2000, 12), (8000,), (2000,))

In [174]:
model = Sequential([
    Dense (64, activation = 'relu', input_shape = (X_train.shape[1],)),
    Dense (32, activation = 'relu'),
    Dense (16, activation = 'relu'),
    Dense (8, activation = 'relu'),
    Dense (1, activation = 'sigmoid')
])

In [156]:
X_train.shape

(8000, 12)

In [89]:
X_train.shape[1] #[1] is for axis=1, means columns that is no of features

12

In [93]:
#model needs shape value as a tuple
type(X_train.shape[1]), type((X_train.shape[1],))

(int, tuple)

In [86]:
X_train.shape[0] #[0] is for axis=0, means rows that is no of obeservations

8000

In [175]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 64)                832       
                                                                 
 dense_9 (Dense)             (None, 32)                2080      
                                                                 
 dense_10 (Dense)            (None, 16)                528       
                                                                 
 dense_11 (Dense)            (None, 8)                 136       
                                                                 
 dense_12 (Dense)            (None, 1)                 9         
                                                                 
Total params: 3585 (14.00 KB)
Trainable params: 3585 (14.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [176]:
#optimizers
import tensorflow
adm = tensorflow.keras.optimizers.Adam(learning_rate = 0.01) #when custom LR is needed
bce = tensorflow.keras.losses.BinaryCrossentropy()

In [177]:
##Compile the model
model.compile(optimizer = adm, loss = "binary_crossentropy", metrics = ['accuracy'])
#if variables created> values will be provided without paraenthesis else in parentheses

In [190]:
##Setup the tensorboard
LogDir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = LogDir, histogram_freq = 1)

In [191]:
##Setup Eralystopping
early_stopping_callbak = EarlyStopping(monitor = 'val_loss', patience = 7, 
restore_best_weights =  True)

In [192]:
##Train the model
history = model.fit(
    X_train, y_train, validation_data = (X_test, y_test), epochs = 100,
    callbacks = [tensorflow_callback, early_stopping_callbak]
)

Epoch 1/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3215 - accuracy: 0.8689 - val_loss: 0.3519 - val_accuracy: 0.8590
Epoch 2/100
250/250 [==============================] - 0s 988us/step - loss: 0.3221 - accuracy: 0.8660 - val_loss: 0.3559 - val_accuracy: 0.8565
Epoch 3/100
250/250 [==============================] - 0s 938us/step - loss: 0.3203 - accuracy: 0.8687 - val_loss: 0.3464 - val_accuracy: 0.8620
Epoch 4/100
250/250 [==============================] - 0s 922us/step - loss: 0.3143 - accuracy: 0.8705 - val_loss: 0.3673 - val_accuracy: 0.8500
Epoch 5/100
250/250 [==============================] - 0s 950us/step - loss: 0.3170 - accuracy: 0.8687 - val_loss: 0.3493 - val_accuracy: 0.8575
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3138 - accuracy: 0.8683 - val_loss: 0.3514 - val_accuracy: 0.8630
Epoch 7/100
250/250 [==============================] - 0s 960us/step - loss: 0.3080 - accuracy: 0.8725 - val_loss: 0.3601 - val_accura

In [187]:
model.save("model.h5")

In [188]:
##Load tensorboard extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [193]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 57939), started 0:42:30 ago. (Use '!kill 57939' to kill it.)